In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [3]:
# for reproducability of the results, lets fix a seed function
np.random.seed(1234)

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
dataset = pd.read_csv('international-airline-passengers.csv', usecols=[1], engine='python',skipfooter=3)

In [12]:
dataset

,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
0,112
1,118
2,132
3,129
4,121
5,135
6,148
7,148
8,136
9,119


In [13]:
dataset = dataset.values
dataset = dataset.astype('float32')

In [14]:
#Normalize the data
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)

In [15]:
dataset

array([[0.01544401],
       [0.02702703],
       [0.05405405],
       [0.04826255],
       [0.03281853],
       [0.05984557],
       [0.08494207],
       [0.08494207],
       [0.06177607],
       [0.02895753],
       [0.        ],
       [0.02702703],
       [0.02123553],
       [0.04247104],
       [0.07142857],
       [0.05984557],
       [0.04054055],
       [0.08687258],
       [0.12741312],
       [0.12741312],
       [0.10424709],
       [0.05598456],
       [0.01930502],
       [0.06949806],
       [0.07915059],
       [0.08880308],
       [0.14285713],
       [0.11389962],
       [0.13127413],
       [0.14285713],
       [0.18339768],
       [0.18339768],
       [0.15444016],
       [0.11196911],
       [0.08108109],
       [0.1196911 ],
       [0.12934363],
       [0.14671814],
       [0.17181468],
       [0.14864865],
       [0.15250966],
       [0.22007722],
       [0.24324325],
       [0.26640925],
       [0.2027027 ],
       [0.16795367],
       [0.13127413],
       [0.173

In [16]:
#spliting the train and test set
train_size = int(len(dataset)*0.67)

In [17]:
print(train_size)

96


In [18]:
test_size = len(dataset)-train_size

In [19]:
print(test_size)

48


In [20]:
train, test = dataset[0:train_size, :], dataset[train_size: len(dataset), :]

In [21]:
print(len(train), len(test))

96 48


In [22]:
# create dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [],[]
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),0]
        dataX.append(a)
        dataY.append(dataset[i+look_back, 0])
    return np.array(dataX), np.array(dataY)

In [23]:
#Reshape dataset X= current time, Y= future time 
look_back=1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [24]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [25]:
print(trainX)

[[[0.01544401]]

 [[0.02702703]]

 [[0.05405405]]

 [[0.04826255]]

 [[0.03281853]]

 [[0.05984557]]

 [[0.08494207]]

 [[0.08494207]]

 [[0.06177607]]

 [[0.02895753]]

 [[0.        ]]

 [[0.02702703]]

 [[0.02123553]]

 [[0.04247104]]

 [[0.07142857]]

 [[0.05984557]]

 [[0.04054055]]

 [[0.08687258]]

 [[0.12741312]]

 [[0.12741312]]

 [[0.10424709]]

 [[0.05598456]]

 [[0.01930502]]

 [[0.06949806]]

 [[0.07915059]]

 [[0.08880308]]

 [[0.14285713]]

 [[0.11389962]]

 [[0.13127413]]

 [[0.14285713]]

 [[0.18339768]]

 [[0.18339768]]

 [[0.15444016]]

 [[0.11196911]]

 [[0.08108109]]

 [[0.1196911 ]]

 [[0.12934363]]

 [[0.14671814]]

 [[0.17181468]]

 [[0.14864865]]

 [[0.15250966]]

 [[0.22007722]]

 [[0.24324325]]

 [[0.26640925]]

 [[0.2027027 ]]

 [[0.16795367]]

 [[0.13127413]]

 [[0.17374519]]

 [[0.17760617]]

 [[0.17760617]]

 [[0.25482625]]

 [[0.25289574]]

 [[0.24131274]]

 [[0.26833975]]

 [[0.3088803 ]]

 [[0.32432434]]

 [[0.25675675]]

 [[0.20656371]]

 [[0.14671814]

In [26]:
print(testX)

[[[0.4073359 ]]

 [[0.3803089 ]]

 [[0.48648646]]

 [[0.47104248]]

 [[0.484556  ]]

 [[0.6138996 ]]

 [[0.6969112 ]]

 [[0.70077217]]

 [[0.57915056]]

 [[0.46911195]]

 [[0.38803086]]

 [[0.44787642]]

 [[0.45559844]]

 [[0.4131274 ]]

 [[0.4980695 ]]

 [[0.47104248]]

 [[0.49999997]]

 [[0.6389961 ]]

 [[0.7471043 ]]

 [[0.7741313 ]]

 [[0.57915056]]

 [[0.492278  ]]

 [[0.3976834 ]]

 [[0.44980696]]

 [[0.49420848]]

 [[0.45945945]]

 [[0.5830116 ]]

 [[0.5637065 ]]

 [[0.61003864]]

 [[0.71042466]]

 [[0.8571429 ]]

 [[0.8783784 ]]

 [[0.69305015]]

 [[0.5849421 ]]

 [[0.4980695 ]]

 [[0.58108103]]

 [[0.6042471 ]]

 [[0.554054  ]]

 [[0.60810804]]

 [[0.6891892 ]]

 [[0.71042466]]

 [[0.8320464 ]]

 [[1.        ]]

 [[0.96911204]]

 [[0.7799227 ]]

 [[0.6891892 ]]]


In [27]:
# Lets create a LSTM(RNN) model
model = Sequential()
model.add(LSTM(4, input_shape = (1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


In [28]:
# fit the model
model.fit(trainX, trainY, batch_size =1, verbose = 2)

Epoch 1/1
 - 1s - loss: 0.0329


In [29]:
#make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [30]:
print(trainPredict)

[[0.10233037]
 [0.10421869]
 [0.10862139]
 [0.10767844]
 [0.10516258]
 [0.109564  ]
 [0.11364422]
 [0.11364422]
 [0.10987813]
 [0.10453334]
 [0.09981184]
 [0.10421869]
 [0.10327462]
 [0.10673521]
 [0.11144814]
 [0.109564  ]
 [0.10642075]
 [0.11395773]
 [0.12052768]
 [0.12052768]
 [0.11677689]
 [0.10893562]
 [0.10295989]
 [0.11113422]
 [0.11270334]
 [0.11427121]
 [0.12302227]
 [0.11834092]
 [0.1211518 ]
 [0.12302227]
 [0.12954381]
 [0.12954381]
 [0.12488975]
 [0.11802825]
 [0.11301702]
 [0.11927854]
 [0.12083977]
 [0.1236451 ]
 [0.12768482]
 [0.1239564 ]
 [0.12457871]
 [0.13540486]
 [0.13908415]
 [0.14274415]
 [0.13263369]
 [0.12706436]
 [0.1211518 ]
 [0.12799491]
 [0.12861477]
 [0.12861477]
 [0.14091666]
 [0.14061159]
 [0.13877824]
 [0.14304823]
 [0.14939885]
 [0.1517995 ]
 [0.1412216 ]
 [0.13325033]
 [0.1236451 ]
 [0.13016267]
 [0.13109012]
 [0.12613294]
 [0.14061159]
 [0.13816607]
 [0.14030637]
 [0.14939885]
 [0.1607009 ]
 [0.15804799]
 [0.14789307]
 [0.13877824]
 [0.13078108]
 [0.13

In [31]:
print(testPredict)

[[0.1645048 ]
 [0.1604069 ]
 [0.17626683]
 [0.17400122]
 [0.17598443]
 [0.19435714]
 [0.2055166 ]
 [0.20602283]
 [0.1895344 ]
 [0.173717  ]
 [0.16158172]
 [0.17057553]
 [0.17172101]
 [0.16537775]
 [0.17795618]
 [0.17400122]
 [0.17823689]
 [0.19778562]
 [0.21200565]
 [0.21541612]
 [0.1895344 ]
 [0.17711255]
 [0.16304578]
 [0.17086223]
 [0.17739402]
 [0.17229243]
 [0.19007453]
 [0.18736346]
 [0.19382557]
 [0.20728326]
 [0.22551486]
 [0.22800535]
 [0.20500922]
 [0.19034417]
 [0.17795618]
 [0.18980458]
 [0.19302621]
 [0.18599823]
 [0.1935594 ]
 [0.20450069]
 [0.20728326]
 [0.22252236]
 [0.24152327]
 [0.23821145]
 [0.2161392 ]
 [0.20450069]]


In [32]:
# Reverse the predicted value to actual values

trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)

trainY = scaler.inverse_transform([trainY])
testY = scaler.inverse_transform([testY])

In [33]:
print(trainPredict, trainY)

[[157.00713]
 [157.98528]
 [160.26588]
 [159.77744]
 [158.47423]
 [160.75415]
 [162.8677 ]
 [162.8677 ]
 [160.91687]
 [158.14827]
 [155.70253]
 [157.98528]
 [157.49625]
 [159.28883]
 [161.73013]
 [160.75415]
 [159.12595]
 [163.0301 ]
 [166.43333]
 [166.43333]
 [164.49043]
 [160.42865]
 [157.33324]
 [161.56754]
 [162.38034]
 [163.19247]
 [167.72554]
 [165.30061]
 [166.75664]
 [167.72554]
 [171.1037 ]
 [171.1037 ]
 [168.6929 ]
 [165.13864]
 [162.54282]
 [165.78627]
 [166.595  ]
 [168.04816]
 [170.14073]
 [168.20943]
 [168.53177]
 [174.13971]
 [176.04558]
 [177.94147]
 [172.70425]
 [169.81935]
 [166.75664]
 [170.30138]
 [170.62244]
 [170.62244]
 [176.99484]
 [176.8368 ]
 [175.88713]
 [178.09898]
 [181.3886 ]
 [182.63214]
 [177.1528 ]
 [173.02367]
 [168.04816]
 [171.42427]
 [171.9047 ]
 [169.33687]
 [176.8368 ]
 [175.57002]
 [176.67871]
 [181.3886 ]
 [187.24306]
 [185.86887]
 [180.60861]
 [175.88713]
 [171.7446 ]
 [175.88713]
 [177.94147]
 [176.52054]
 [181.85558]
 [182.16647]
 [182.32176]

In [34]:
print(testPredict, testY)

[[189.21349]
 [187.09079]
 [195.30621]
 [194.13263]
 [195.15993]
 [204.67699]
 [210.45761]
 [210.71983]
 [202.17882]
 [193.9854 ]
 [187.69934]
 [192.35812]
 [192.95149]
 [189.66568]
 [196.1813 ]
 [194.13263]
 [196.3267 ]
 [206.45296]
 [213.81892]
 [215.58554]
 [202.17882]
 [195.74431]
 [188.45772]
 [192.50664]
 [195.89009]
 [193.24748]
 [202.4586 ]
 [201.05428]
 [204.40164]
 [211.37273]
 [220.8167 ]
 [222.10677]
 [210.19478]
 [202.59828]
 [196.1813 ]
 [202.31877]
 [203.98758]
 [200.34709]
 [204.26376]
 [209.93137]
 [211.37273]
 [219.26657]
 [229.10905]
 [227.39354]
 [215.9601 ]
 [209.93137]] [[301.00001152 355.999993   348.00001085 355.00000681 421.9999924
  465.0000161  466.99998848 403.99999397 346.99999379 304.99998716
  335.99999132 339.99999783 317.99999288 362.00000277 348.00001085
  362.99998896 434.99999813 491.00002756 505.00001948 403.99999397
  359.00001333 310.00001073 337.00000838 359.99999952 342.00000109
  406.0000281  395.99998094 420.00002002 471.99998119 548.00004318


In [35]:
# Calculate RMSE
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print('Train : %.2f RMSE' % (trainScore))

Train : 72.86 RMSE


In [36]:
testScore= math.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
print('Test : %.2f RMSE' % (testScore))

Test : 223.28 RMSE
